In [11]:
import sys
!{sys.executable} -m pip install pmdarima

   ---------------------------------------- 0.0/614.7 kB ? eta -:--:--
   -- ------------------------------------- 41.0/614.7 kB ? eta -:--:--
   --- ----------------------------------- 51.2/614.7 kB 650.2 kB/s eta 0:00:01
   ---- ---------------------------------- 71.7/614.7 kB 393.8 kB/s eta 0:00:02
   -------- ----------------------------- 133.1/614.7 kB 714.4 kB/s eta 0:00:01
   --------- ---------------------------- 153.6/614.7 kB 573.4 kB/s eta 0:00:01
   ----------- -------------------------- 184.3/614.7 kB 619.5 kB/s eta 0:00:01
   ------------ ------------------------- 204.8/614.7 kB 655.1 kB/s eta 0:00:01
   ----------------- -------------------- 276.5/614.7 kB 680.9 kB/s eta 0:00:01
   ------------------ ------------------- 297.0/614.7 kB 733.3 kB/s eta 0:00:01
   ------------------ ------------------- 297.0/614.7 kB 733.3 kB/s eta 0:00:01
   ------------------ ------------------- 297.0/614.7 kB 733.3 kB/s eta 0:00:01
   ------------------ ------------------- 297.0/614.7 kB 


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\saira\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from pmdarima.arima import ARIMA, auto_arima
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.simplefilter("ignore")

In [14]:
import pandas as pd
import numpy as np
from pmdarima.arima import ARIMA
import warnings

warnings.filterwarnings("ignore")

# Load dataset
dataset = pd.read_csv("C:\\Users\\saira\\Desktop\\arima_lstm_model\\train_dev_test\\new_asset_before_arima.csv")
dataset = dataset.loc[:, ~dataset.columns.str.contains('Unnamed')]

residuals = []

orders = [(1, 1, 0), (0, 1, 1), (1, 1, 1), (2, 1, 1), (2, 1, 0)]

for idx, row in enumerate(dataset.values):
    series = pd.Series(row)

    best_model = None
    best_aic = np.inf

    for order in orders:
        try:
            model = ARIMA(order=order, suppress_warnings=True)
            fitted = model.fit(series)
            if fitted.aic() < best_aic:
                best_model = fitted
                best_aic = fitted.aic()
        except:
            continue

    if best_model is None:
        print(f"{idx + 1} FATAL ERROR: All models failed for this series.")
        break

    # Generate in-sample predictions
    predictions = best_model.predict_in_sample()
    predictions = [np.mean(predictions)] + list(predictions[:-1])  # Shift by 1
    residual = series - predictions
    residuals.append(residual.values)

# Save residuals
residual_df = pd.DataFrame(residuals)
residual_df.to_csv("C:\\Users\\saira\\Desktop\\arima_lstm_model\\train_dev_test\\new_asset_after_arima.csv", index=False)
